## <h1><u>Preparing Data in Customer Table</u></h1>

### 1. Understanding & Investigating the Data 

### 2. Mixed Data Type Check 

### 3. Renaming Columns 

### 4. Missing Observations Check (NaN) 

### 5. Duplicate Check 

### 6. Joining Orders/Prods Table with Customers  

In [1]:
## importing libraries & data 
import pandas as pd 
import numpy as np 
import os 

path = r'/Users/aahamoustafa/Desktop/Data Projects/Instacart Basket Analysis  12-2024'

df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_aggregations.pkl'))

df_dept = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'), index_col = False)

df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

## 1. Understanding & Investigating customer data

In [3]:
## understanding the new customer data 

pd.set_option('display.float_format', '{:.2f}'.format) ## remove scientific notation

df_cust.describe()

,user_id,Age,n_dependants,income
count,206209.00,206209.00,206209.00,206209.00
mean,103105.00,49.50,1.50,94632.85
std,59527.56,18.48,1.12,42473.79
min,1.00,18.00,0.00,25903.00
25%,51553.00,33.00,0.00,59874.00
50%,103105.00,49.00,1.00,93547.00
75%,154657.00,66.00,3.00,124244.00
max,206209.00,81.00,3.00,593901.00


In [15]:
df_cust.sort_values(by = 'income', ascending = False)


,user_id,first_name,surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
21726,58596,Thomas,Cohen,Male,Washington,75,5/6/2017,2,married,593901
202655,98580,nan,Patterson,Male,Washington,43,3/12/2020,2,married,592409
81583,103702,Kathryn,Mclaughlin,Female,Louisiana,55,4/14/2018,3,married,591089
111950,187680,Catherine,Church,Female,Wyoming,43,10/7/2018,2,married,590790
167037,189861,Pamela,Jordan,Female,Pennsylvania,47,8/21/2019,0,single,584097
...,...,...,...,...,...,...,...,...,...,...
67441,151083,Katherine,Rush,Female,Montana,32,1/24/2018,2,married,25955
169181,183978,Frances,Nixon,Female,Montana,39,9/2/2019,3,married,25941
194354,162338,nan,Page,Female,California,21,1/25/2020,1,living with parents and siblings,25937
105031,193849,Patricia,Hart,Female,West Virginia,34,8/27/2018,2,married,25911


#### Looks like there are is a really high income outside of the mean/median range, we will need to investigate further using some charts

In [5]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


## 2. Mixed data type check

In [ ]:
## any mixed type columns?

for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

#### First name column has mixed type due to its NaN values 

In [7]:
## changing First Name Column to string data type 

df_cust['first_name'] = df_cust['first_name'].astype('str')

## 3. Renaming Columns 

In [5]:
## renaming Some column names 

df_cust = df_cust.rename(columns = {'Surnam' : 'surname','STATE' : 'State', 'First Name' : 'first_name'})
df_cust.head()

,user_id,first_name,surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 4. Checking for Missing Observations 

In [ ]:
## finding missing observations 

df_cust.isnull().sum()

In [ ]:
### looking at those missing values

df_nan_cust = df_cust[df_cust['first_name'].isnull() == True]

df_nan_cust

#### We have 11259 rows with missing first names, but we will still use the data as it contains other important information related to the user. 

## 5. Duplicate Checks 

In [15]:
## finding duplicates 

df_dups_cust = df_cust[df_cust.duplicated()]

df_dups_cust

,user_id,first_name,surname,Gender,State,Age,date_joined,n_dependants,fam_status,income


#### No duplicates!

## 6. Merging Order/products Table with Customers Table

In [ ]:
## checking on data type for ( user_id) as we are joining on it 

df_ords_prods.info()

## dropping some columns 

df_ords_prods.drop(columns='_merge', inplace=True)


In [11]:
## merging tables 

df_merged = df_ords_prods.merge(df_cust, on=['user_id'], indicator='merge_status')

In [19]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 32 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   order_id                       int64   
 1   user_id                        int64   
 2   order_number                   int64   
 3   order_day_of_week              int64   
 4   order_hour_of_day              int64   
 5   days_since_prior_order         float64 
 6   product_id                     int64   
 7   add_to_cart_order              int64   
 8   reordered                      int64   
 9   product_name                   object  
 10  aisle_id                       int64   
 11  department_id                  int64   
 12  prices                         float64 
 13  price_range_loc                object  
 14  Busiest_day                    object  
 15  Busiest_period_of_day          object  
 16  max_order                      int64   
 17  loyalty_flag             

In [17]:
## checking on merge flag with inner join 

df_merged['merge_status'].value_counts()

merge_status
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [13]:
## dropping the merge status column 

df_merged.drop(columns='merge_status', inplace=True)

#### Data is Clean and all users present in customer table are present in the original ords-prods table 

## 7. Exporting the Data

In [24]:
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_cust.pkl'))